In [0]:
#import 20 news groups data
from sklearn.datasets import fetch_20newsgroups 


In [2]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
x = len(twenty_train.target_names) 
print("\n The number of categories:",x) 
print("\n The %d Different Categories of 20Newsgroups\n" %x) 
i=1 
for cat in twenty_train.target_names:     
    print("Category[%d]:" %i,cat)     
    i=i+1 
    
print("\n Length of training data is",len(twenty_train.data)) 
print("\n Length of file names is ",len(twenty_train.filenames)) 
 
print("\n The Content/Data of First File is :\n") 
 
print(twenty_train.data[0])


 The number of categories: 20

 The 20 Different Categories of 20Newsgroups

Category[1]: alt.atheism
Category[2]: comp.graphics
Category[3]: comp.os.ms-windows.misc
Category[4]: comp.sys.ibm.pc.hardware
Category[5]: comp.sys.mac.hardware
Category[6]: comp.windows.x
Category[7]: misc.forsale
Category[8]: rec.autos
Category[9]: rec.motorcycles
Category[10]: rec.sport.baseball
Category[11]: rec.sport.hockey
Category[12]: sci.crypt
Category[13]: sci.electronics
Category[14]: sci.med
Category[15]: sci.space
Category[16]: soc.religion.christian
Category[17]: talk.politics.guns
Category[18]: talk.politics.mideast
Category[19]: talk.politics.misc
Category[20]: talk.religion.misc

 Length of training data is 11314

 Length of file names is  11314

 The Content/Data of First File is :

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there coul

In [3]:
#this is an optional. just to see the contents of file, im printing first 5 files data.
print("\n The Contents/Data of First 5 Files is in Training Data :\n") 
 
for i in range(0,5):     
    print("\n FILE NO:%d \n"%(i+1))     
    print(twenty_train.data[i])


 The Contents/Data of First 5 Files is in Training Data :


 FILE NO:1 

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----






 FILE NO:2 

From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvf

In [4]:
#Out of 20 categories, considering only four categories (you can use any categories, programmer choice)
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med'] 
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42) 
print("\n Reduced Target Names:\n",twenty_train.target_names) 
print("\n Reduced Target Length:\n", len(twenty_train.data)) 



 Reduced Target Names:
 ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

 Reduced Target Length:
 2257


In [5]:
#optional, just to see the content of first file
print("\nFirst Document : ",twenty_train.data[0])


First Document :  From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



In [6]:
#Extracting features from text files( Word Occurrences) 

from sklearn.feature_extraction.text import CountVectorizer 
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(twenty_train.data) 

print("\n(Target Length , Distinct Words):",X_train_counts.shape)  
print("\n Frequency of the word algorithm:",count_vect.vocabulary_.get('algorithm')) 

from sklearn.feature_extraction.text import TfidfTransformer 
tfidf_transformer = TfidfTransformer() 
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts) 
X_train_tfidf.shape


(Target Length , Distinct Words): (2257, 35788)

 Frequency of the word algorithm: 4690


(2257, 35788)

In [0]:
#Training a classifier 
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [8]:
#Predicting the Outcome 

docs_new = ['God is love', 'OpenGL on the GPU is fast'] 
X_new_counts = count_vect.transform(docs_new) 
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf) 
 
for doc, category in zip(docs_new, predicted):     
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [9]:
#Building a pipeline 
#In order to make the vectorizer => transformer => classifier easier to work with,
# scikit-learn provides a Pipeline class that behaves like a compound classifier: 

from sklearn.pipeline import Pipeline 
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])

text_clf.fit(twenty_train.data, twenty_train.target) 

#Evaluation of the performance on the test set 

import numpy as np 
twenty_test = fetch_20newsgroups(subset='test',categories=categories, shuffle =True, random_state=42) 
docs_test = twenty_test.data 
predicted = text_clf.predict(docs_test) 
np.mean(predicted == twenty_test.target) 

from sklearn import metrics 
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names)) 

                        precision    recall  f1-score   support

           alt.atheism       0.97      0.60      0.74       319
         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.97      0.81      0.88       396
soc.religion.christian       0.65      0.99      0.78       398

              accuracy                           0.83      1502
             macro avg       0.89      0.82      0.83      1502
          weighted avg       0.88      0.83      0.84      1502

